# COVbat TOP and StrokeMRI  vs. Insight46 harmonized datasets

Note this must be run in the `covbat` environment. Environment file inside vendor/covbat

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

import patsy

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.vendor.covbat.covbat as covbat

impo##rt data

In [ ]:
# Datasets for this work
Insight = pd.read_csv('../new_data/TrainingDataComplete_Insight46.csv')

MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')


In [ ]:
Insight.head(3)

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)

TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
Insight = Insight.drop(['ID'],axis = 1)
Insight['Site'] = 4
Insight.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 2
TOPMRI.head(3) 

In [ ]:
TOPMRI.tail(3)

In [ ]:
to_be_harmonized_or_covar = [
    'Age', 'Sex', 'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV','DeepWM_B_CoV',
     'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'LD', 'PLD','DeepWM_B_CBF',
       'Labelling', 'Readout', 'M0','TotalGM_B_CoV',
       'TotalGM_B_CBF',
]
not_harmonized= ['GM_vol', 'WM_vol', 'CSF_vol','GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
                'LD', 'PLD', 'Labelling',
       'Readout', 'M0','DeepWM_B_CoV','DeepWM_B_CBF',]
TOPMRI_semi_features = TOPMRI.drop(to_be_harmonized_or_covar,axis=1)
Insight_semi_features = Insight.drop(to_be_harmonized_or_covar,axis=1)

TOPMRI = TOPMRI.drop(not_harmonized,axis=1)
Insight = Insight.drop(not_harmonized,axis=1)

In [ ]:
TOPMRI.columns

In [ ]:
#datasets1 = [TOPMRI, Insight,]

In [ ]:
# datasets = []
# for myset in datasets1:
#     myset = myset.drop(['LD', 'PLD', 'Labelling', 'Readout', 'M0'], axis=1)
#     datasets.append(myset)#print(myset.columns)

In [ ]:
TOPMRIInsight = pd.concat([TOPMRI, Insight])

In [ ]:
TOPMRIInsight.head(3)

In [ ]:
phenoTOPMRIInsight = TOPMRIInsight[['participant_id','Age', 'Sex', 'Site']]
phenoTOPMRIInsight = phenoTOPMRIInsight.set_index('participant_id')
phenoTOPMRIInsight.head(3)

In [ ]:
dat_TOPMRIInsight = TOPMRIInsight.set_index('participant_id')
dat_TOPMRIInsight = dat_TOPMRIInsight.T 

In [ ]:
dat_TOPMRIInsight.head(3)

In [ ]:
phenoTOPMRIInsight.head(3)

In [ ]:
modZ = patsy.dmatrix("~ Age + Sex", phenoTOPMRIInsight, return_type="dataframe")
modZ.head(3)

In [ ]:
dat_TOPMRIInsight

In [ ]:
covbatTOPMRIInsight = covbat.combat(dat_TOPMRIInsight.tail(10), phenoTOPMRIInsight['Site'],  model=modZ, numerical_covariates ="Age")

In [ ]:
covbatTOPMRIInsight

In [ ]:
covbatTOPMRIInsight = covbatTOPMRIInsight[2:]

covbatTOPMRIInsight 

In [ ]:
dat_TOPMRIInsight.head(3)

In [ ]:
covbatTOPMRIInsight = pd.concat([dat_TOPMRIInsight.head(3), covbatTOPMRIInsight])
covbatTOPMRIInsight = covbatTOPMRIInsight.T
covbatTOPMRIInsight = covbatTOPMRIInsight.reset_index()

In [ ]:
covbatTOPMRIInsight

In [ ]:
TOPMRI_adjusted = covbatTOPMRIInsight[covbatTOPMRIInsight['Site'] == 2]
Insight_adjusted   = covbatTOPMRIInsight[covbatTOPMRIInsight['Site'] == 4] 

In [ ]:
TOPMRI_adjusted = TOPMRI_adjusted.drop('Site', axis=1)
TOPMRI_adjusted.head(3)

In [ ]:
Insight_adjusted = Insight_adjusted.drop('Site', axis=1)
Insight_adjusted.tail(3)

In [ ]:
TOPMRI_adjusted = TOPMRI_adjusted.merge(TOPMRI_semi_features,on='participant_id')
Insight_adjusted = Insight_adjusted.merge(Insight_semi_features,on='participant_id')

In [ ]:
Insight_adjusted.columns

In [ ]:
Insight_adjusted = Insight_adjusted.drop('Site', axis=1)
TOPMRI_adjusted = TOPMRI_adjusted.drop('Site', axis=1)

In [ ]:
TOPMRI_adjusted.to_csv('harm_results/covbat/topmri_covbat_a_insight.csv')
Insight_adjusted.to_csv('harm_results/covbat/insight_covbat_a_topmri.csv')

In [ ]:
Insight_adjusted.shape